In [ ]:
!pip install transformers timm einops -q


In [ ]:
!wget -O image.jpg https://live.staticflickr.com/4071/4245769620_e33561e242_z.jpg

--2024-03-14 18:07:21--  https://live.staticflickr.com/4071/4245769620_e33561e242_z.jpg
Resolving live.staticflickr.com (live.staticflickr.com)... 52.84.158.78, 2600:9000:20be:8600:0:5a51:64c9:c681, 2600:9000:20be:a800:0:5a51:64c9:c681, ...
Connecting to live.staticflickr.com (live.staticflickr.com)|52.84.158.78|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [image/jpeg]
Saving to: ‘image.jpg’

image.jpg               [ <=>                ]  31.97K  --.-KB/s    in 0.001s  

2024-03-14 18:07:22 (28.4 MB/s) - ‘image.jpg’ saved [32741]



In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from PIL import Image
import torch

model_id = "vikhyatk/moondream2"
revision = "2024-03-05"


# Check if CUDA (GPU support) is available and then set the device to GPU or CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the model and tokenizer
model = AutoModelForCausalLM.from_pretrained(model_id, trust_remote_code=True, revision=revision).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_id, revision=revision)



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/318 [00:00<?, ?B/s]

configuration_moondream.py:   0%|          | 0.00/3.43k [00:00<?, ?B/s]

moondream.py:   0%|          | 0.00/3.20k [00:00<?, ?B/s]

modeling_phi.py:   0%|          | 0.00/49.4k [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


# Image


In [ ]:

image = Image.open('image.jpg') #path of the input image

enc_image = model.encode_image(image).to(device)

# Generate the answer
answer = model.answer_question(enc_image, "Write a joke about this image", tokenizer)

# If the output is a tensor, move it back to CPU for further operations like print
if isinstance(answer, torch.Tensor):
    answer = answer.cpu()

print(answer)


Three apples are arranged side by side in this image, with a single leaf attached to each one. The background is white, and the apples appear to be the main focus of the image.


# GIF

In [ ]:
# from google.colab import files
# uploaded = files.upload()

from PIL import Image


In [ ]:
import locale;locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
!wget -O tenor.gif https://media1.tenor.com/m/98GZJjf3CeUAAAAC/bake-pie.gif

--2024-03-14 18:23:30--  https://media1.tenor.com/m/98GZJjf3CeUAAAAC/bake-pie.gif
Resolving media1.tenor.com (media1.tenor.com)... 108.177.98.95, 74.125.197.95, 74.125.135.95, ...
Connecting to media1.tenor.com (media1.tenor.com)|108.177.98.95|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3408429 (3.2M) [image/gif]
Saving to: ‘tenor.gif’

tenor.gif           100%[===================>]   3.25M  --.-KB/s    in 0.02s   

2024-03-14 18:23:30 (151 MB/s) - ‘tenor.gif’ saved [3408429/3408429]



In [ ]:
# Load the GIF file
gif_path = 'tenor.gif'  # Replace with your gif path
gif = Image.open(gif_path)

frame_count = 0
answers = []

while True:
    try:
        # Seek to the next frame
        gif.seek(frame_count)

        # Process every 5th frame
        if frame_count % 10 == 0:
            # Convert the frame to a format compatible with the model
            frame = gif.convert('RGB')

            # Process the frame with the model
            enc_image = model.encode_image(frame).to(device)
            answer = model.answer_question(enc_image, "Give me the description of the image and also read the text on the image", tokenizer)
            if isinstance(answer, torch.Tensor):
                answer = answer.cpu()

            # Append the answer
            answers.append(answer)

        frame_count += 1
    except EOFError:
        # Exit the loop when the end of the GIF is reached
        break

# Concatenate and print all answers
all_answers = '\n'.join(answers)
print(all_answers)

A wooden table is the central focus of this image, with a bowl containing a liquid and a brush placed on top. A glass bowl with a food item is also present. The image features some text at the bottom.
A table is depicted in this image, with a glass bowl containing a pie on top. A brush is placed in the bowl, and a person's hand is visible on the right side of the image. The table is brown in color.
A table is depicted in this image, with a bowl of yellow substance and a wooden object on its surface. A glass bowl containing food with text is also present. A white object with a handle is situated nearby.
A table is depicted in this image, with a glass bowl containing a pie on top. A brush is placed in the bowl, and a person's hand is visible on the right side of the image. The table is brown in color.


#TODO: Ask an LLM to summarize!